In [ ]:
!pip install turtle==0.0.2
!pip install openai
!brew install python-tk

In [2]:
import openai
import json

openai.api_key = <open_ai_api_key>

In [4]:
def function_calling(query):
    functions = [
        {
            "name": "forward",
            "description": "Move forward by the specified distance, in the same direction as previously showing.",
            "parameters": {
                "type": "object",
                "properties": {
                    "distance": {
                        "type": "number",
                        "description": "A number that specifies the distance to move forward "
                    }
                 },
                },
                "required": ["distance"],
        },

        {
            "name": "circle",
            "description": "Draw a circle with given radius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "radius": {
                        "type": "number",
                        "description": "return the radius in the query"
                    }
                 },
                },
                "required": ["radius"],
            },
        {
            "name": 'python',
            "description": 'Not implemented.',
            "parameters": {
                "type": 'object',
                "properties": {}
            }
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "system", "content": query + ". You don’t have a python function available, never attempt to call python."}],
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message

In [5]:
import turtle

# Define a function to call other functions dynamically
def call_function(name, args):
    print("name: " +  name + " args: " + str(args))
    # Get a reference to the function based on its name
    func = getattr(turtle, name)
    
    # Call the function with the arguments
    if args is None:
        result = func()
    else:
        result = func(args)
    
    # Return the result
    return result

In [22]:
import json

def call_function_from_query(query):
    # Call the function specified in the query
    response = function_calling(query)
    function_name = response['function_call']['name']
    arguments = json.loads(response['function_call']['arguments'])
    first_value = next(iter(arguments.values()))
    
    call_function(function_name, first_value)
    
    # Call the 'bye' function to close the program
    call_function('bye', None)

In [23]:
query_1 = "darw a circle with radius 10"
query_2 = "move forward 100"

In [25]:
call_function_from_query(query_1)


name: circle args: 10
name: bye args: None
